In [ ]:
import pandas as pd
import requests


from database_functions import get_config
from datetime import datetime
from sqlalchemy import create_engine

In [ ]:
#GLOBALS
TABLE_NAME_TARGET='crypto_timeseries'
CONFIG_DB = get_config(filename="database.ini", section="crypto")
CONFIG_API = get_config(filename="database.ini", section="api")

API_URL="https://rest.coincap.io/v3/assets?apiKey={api_key}"
HEADER_API = {
            "Content-Type":"application/json",
            "Accept-Encoding":"deflate" 
        }

In [ ]:
##----> ETRACT <----- ##
#API query
response = requests.get(url= API_URL.format(api_key=CONFIG_API['api_key']),
                        headers=HEADER_API)

In [ ]:
##----> TRANSFORM <----- ##
#API json response to pandas Dataframe
responseData=response.json()
df = pd.json_normalize(data=responseData, record_path='data')

#insert timestamp
current_timestamp = datetime.now()
current_timestamp.strftime('%d-%m-%Y %H:%M:%S')
df['timestamp'] = [current_timestamp]*df.shape[0]

#rename columns to lowercase
rename_cols_dict={c:c.lower() for c in df.columns.tolist()}
df.rename(columns=rename_cols_dict, inplace=True)

#drop columns with tokens*
df = df.loc[:, ~df.columns.str.startswith("tokens.")]


In [ ]:
##----> LOAD <----- ##
#load data into postresql database
conn_string = f"postgresql://{CONFIG_DB['user']}:{CONFIG_DB['password']}@{CONFIG_DB['host']}/{CONFIG_DB['database']}"

engine = create_engine(conn_string)  

df.to_sql(
            name=TABLE_NAME_TARGET,
            con=engine,
            if_exists='append',
            index=True
        )
